In [2]:
import os
import pandas as pd
import numpy as np

# 設定輸出資料夾路徑
output_folder = r'G:\works\Data_archive\8f_behav\open_field\2023April\480p\dlc_output'

# 取得所有.h5檔案路徑
file_paths = [os.path.join(output_folder, f) for f in os.listdir(output_folder) if f.endswith('.h5')]

# 儲存所有檔案的距離和速度
results = {}

# 迴圈處理每個檔案
for file_path in file_paths:
    # 讀取檔案
    df = pd.read_hdf(file_path)
    
    # 選取所有likelihood欄
    likelihood_cols = [col for col in df.columns if 'likelihood' in col]
    
    # 篩選所有部位中likelihood小於0.6的frame
    mask = (df[likelihood_cols] >= 0.6).all(axis=1)
    df_filtered = df[mask]
    
    # 取出spine1和spine2的座標
    spine1_coords = df_filtered['DLC_resnet50_OpenFieldApr28shuffle1_86000']['spine1'][['x', 'y']]
    spine2_coords = df_filtered['DLC_resnet50_OpenFieldApr28shuffle1_86000']['spine2'][['x', 'y']]
    
    # 計算中點的座標
    midpoint_coords = (spine1_coords + spine2_coords) / 2
    
    # 計算每個時間點的移動距離
    distances = np.sqrt(((midpoint_coords - midpoint_coords.shift()) ** 2).sum(axis=1))
    
    # 計算總移動距離
    total_distance = int(distances.sum() / 480 * 40)
    
    # 計算平均速度
    avg_speed = round(total_distance / 1200, 2)
    
    # 計算相鄰frame中點座標的變化量
    coord_diff = np.abs(np.diff(midpoint_coords, axis=0))

    # 找出變化量大於設定 pixel的frame，並計算符合條件的frame數量 (計算moving time)
    num_still_frames = np.sum(np.all(coord_diff >= 0.24, axis=1))
    moving_time = num_still_frames*40/1000
    
    # 儲存結果
    file_name = os.path.basename(file_path)
    prefix = 'openfield_'
    suffix = 'DLC_resnet50_OpenFieldApr28shuffle1_70000.h5'
    trimmed_name = file_name[len(prefix):len(file_name)-len(suffix)]
    results[trimmed_name] = {'distance': total_distance, 'speed': avg_speed, 'moving_time' : moving_time}
    
    

# 將結果輸出到 excel 檔案中
df = pd.DataFrame.from_dict(results, orient='index')
df.index.name = '檔名'
df.columns = ['距離', '速度', '運動時間']
df.to_excel('output.xlsx')

# 輸出結果
print(df)

              距離    速度    運動時間
檔名                            
0420_0904A  5291  4.41  649.20
0420_0904B  4601  3.83  546.28
0420_0904C  6199  5.17  600.80
0420_0904D  5480  4.57  559.68
0420_1213A  6244  5.20  615.36
0420_1213B  5327  4.44  545.04
0420_1213C  4661  3.88  566.00
0420_1213D  6633  5.53  640.32
0420_1213E  6639  5.53  664.92
0420_1219A  4699  3.92  491.60
0420_1219B  3861  3.22  517.12
0420_1219D  6098  5.08  617.36
0420_1219E  5718  4.76  552.44
0420_G210   5622  4.68  517.32
0420_G221   8101  6.75  640.56
0424_0904A  5340  4.45  566.68
0424_0904B  4364  3.64  482.68
0424_0904C  4297  3.58  529.56
0424_0904D  3736  3.11  494.68
0424_1213A  3271  2.73  364.48
0424_1213B  3947  3.29  422.92
0424_1213C  3092  2.58  369.76
0424_1213D  4087  3.41  441.24
0424_1213E  5757  4.80  593.20
0424_1219A  4422  3.69  496.08
0424_1219B  3964  3.30  503.76
0424_1219D  4233  3.53  558.84
0424_1219E  4274  3.56  521.24
0424_G210   6314  5.26  554.92
0424_G221   6982  5.82  574.36
